Import usage

In [54]:
# Loading the required libraries (only these are necessary)
import pandas as pd
import numpy as np
from scipy.stats import norm

Loading dataset(relative) and cleaning data before use.

In [55]:
# Importing raw data via a relative path
df = pd.read_csv('../data/transaction_data.csv')
# Standardize column names for easier access and understanding
df_clean = df.rename(columns={
    'Lead Time': 'leadTime',
    'Order-Quantity': 'order_quantity',
    'Manufacturing Site': 'manufacturing_site'
})
df_clean = df_clean[df_clean['unit_price'] >= 0]
df_clean = df_clean[df_clean['order_quantity'] >= 0]
df_clean = df_clean[df_clean['leadTime'] < 2800]
# Drop rows where essential identifiers or categories are missing
df_clean = df_clean.dropna(subset=['sku_number', 'inventory_type', 'stocking_type'])
df_filtered = df_clean[
    (df_clean['inventory_type'] == 'FG') & 
    (df_clean['stocking_type'] == 'MTS')
].copy()
# Print the shape (rows, columns) of the dataframes to verify cleaning/filtering
print(f"Cleaned data shape: {df_clean.shape}")
print(f"Filtered (FG, MTS) data shape: {df_filtered.shape}")

Cleaned data shape: (367271, 9)
Filtered (FG, MTS) data shape: (140394, 9)


Question 1 is featured below, aggregating order quantities, getting average lead time, and lastly displaying the transformed dataset.

In [56]:
df_agg = df_filtered.groupby('sku_number').agg(
    # Calculate min, max, mean, median, variance, and std dev for order quantity
    qty_min=('order_quantity', 'min'),
    qty_max=('order_quantity', 'max'),
    qty_avg=('order_quantity', 'mean'),
    qty_median=('order_quantity', 'median'),
    qty_var=('order_quantity', 'var'),
    qty_std=('order_quantity', 'std'),
    
    avg_lead_time=('leadTime', 'mean')
).reset_index()
# Display the first 5 rows of the new aggregated DataFrame
print("\nAggregated SKU Statistics (Head):")
df_agg.head()


Aggregated SKU Statistics (Head):


,sku_number,qty_min,qty_max,qty_avg,qty_median,qty_var,qty_std,avg_lead_time
0,0019425F,38,180,98.726005,98.0,372.534060,19.301141,28.0
1,00DCA10C,0,28,9.834437,9.0,42.245740,6.499672,14.0
2,01FE860A,1,27,9.590909,9.0,41.689218,6.456719,14.0
3,0336A033,40,181,100.386293,100.0,418.525642,20.457899,28.0
4,058CD6C0,34,174,98.641903,98.0,406.347015,20.158051,28.0


Question 2 is featured below, defining service levels and getting the z-scores necessary for safety stock calculations.

In [57]:
# Define the target cycle service levels (CSL) as probabilities
service_levels = [0.75, 0.90, 0.95]
z_scores = {sl: norm.ppf(sl) for sl in service_levels}

# Print the calculated Z-scores formatted to 4 decimal places
print(f"Z-score for 75% SL: {z_scores[0.75]:.4f}")
print(f"Z-score for 90% SL: {z_scores[0.90]:.4f}")
print(f"Z-score for 95% SL: {z_scores[0.95]:.4f}")

Z-score for 75% SL: 0.6745
Z-score for 90% SL: 1.2816
Z-score for 95% SL: 1.6449


The sqrt(L) factor gets used to get the last lead time for the SS to calculate with, which is necessary to get our final results for the 3 different requirements/z-scores.

In [58]:
df_ss = df_agg.copy()

df_ss['sqrt_lead_time'] = np.sqrt(df_ss['avg_lead_time'])
# Calculating the raw safety stock based on the formula SS = Z * std_dev_demand_per_period * sqrt(lead_time_in_periods)
df_ss['ss_75_raw'] = z_scores[0.75] * df_ss['qty_std'] * df_ss['sqrt_lead_time']
df_ss['ss_90_raw'] = z_scores[0.90] * df_ss['qty_std'] * df_ss['sqrt_lead_time']
df_ss['ss_95_raw'] = z_scores[0.95] * df_ss['qty_std'] * df_ss['sqrt_lead_time']

df_ss['ss_75'] = np.ceil(df_ss['ss_75_raw']).fillna(0)
df_ss['ss_90'] = np.ceil(df_ss['ss_90_raw']).fillna(0)
df_ss['ss_95'] = np.ceil(df_ss['ss_95_raw']).fillna(0)
# Display the final safety stock values for the first 5 SKUs
print("SKU Safety Stock Calculations (Head):")
df_ss[['sku_number', 'ss_75', 'ss_90', 'ss_95']].head()

SKU Safety Stock Calculations (Head):


,sku_number,ss_75,ss_90,ss_95
0,0019425F,69.0,131.0,168.0
1,00DCA10C,17.0,32.0,41.0
2,01FE860A,17.0,31.0,40.0
3,0336A033,74.0,139.0,179.0
4,058CD6C0,72.0,137.0,176.0


We use nlargest().

In [59]:
# Find the SKU with the single largest safety stock at the 95% service level
sku_largest_ss = df_ss.nlargest(1, 'ss_95')
print("SKU with the largest safety stock (95% SL):")
sku_largest_ss[['sku_number', 'ss_95']]

SKU with the largest safety stock (95% SL):


,sku_number,ss_95
89,528EC5AC,337.0


We use nsmallest(). Note that a value of 0 is possible if an SKU had 0 variance or only one transaction.

In [60]:
# Find the SKU with the single smallest safety stock at the 95% service level
sku_smallest_ss = df_ss.nsmallest(1, 'ss_95')
print("SKU with the smallest safety stock (95% SL):")
sku_smallest_ss[['sku_number', 'ss_95']]

SKU with the smallest safety stock (95% SL):


,sku_number,ss_95
176,9ED24ECA,32.0


In [61]:
avg_ss_95 = df_ss['ss_95'].mean()
print(f"The average safety stock (95% SL) across all SKUs is: {avg_ss_95:.2f} units")

The average safety stock (95% SL) across all SKUs is: 105.29 units


The side-quest is featured below, starting with mstats being imported. cal_mquantiles, SKU order quantity calculations, and non-parametric safety stock calculations will follow.

In [62]:
# Import mstats (masked statistics) from scipy.stats
from scipy.stats import mstats

print("Imported scipy.stats.mstats for non-parametric calculations.")

Imported scipy.stats.mstats for non-parametric calculations.


This function below uses scipy.stats.mstats to calculate the 75th, 90th, and 95th empirical quantiles for a data series. It's designed to work perfectly with pandas' .apply() method by returning a new series

In [63]:
# Define a helper function that we can use with pandas' .apply() method
def calc_mquantiles(series):
    """
    Calculates the 75th, 90th, and 95th empirical quantiles
    for a pandas series using mstats.
    """
    quantiles = mstats.mquantiles(series, prob=[0.75, 0.90, 0.95])
    
    return pd.Series(quantiles, index=[0.75, 0.90, 0.95])

print("Helper function calc_mquantiles defined.")

Helper function calc_mquantiles defined.


Group by SKU, apply the calc_mquantiles helper function to find the 75th, 90th, and 95th percentile values from the actual data, and then rename the columns for clarity.

In [64]:
print("Calculating empirical quantiles for each SKU's order_quantity...")

df_quantiles_np = df_filtered.groupby('sku_number')['order_quantity'].apply(calc_mquantiles).unstack()
# Rename the columns from the probability (e.g., 0.75) to a descriptive name (e.g., q_np_75)
df_quantiles_np = df_quantiles_np.rename(columns={
    0.75: 'q_np_75',
    0.90: 'q_np_90',
    0.95: 'q_np_95'
})

print("Quantile calculation complete. Head of quantiles:")
df_quantiles_np.head()

Calculating empirical quantiles for each SKU's order_quantity...
Quantile calculation complete. Head of quantiles:


,q_np_75,q_np_90,q_np_95
sku_number,,,
0019425F,112.0,124.0,131.00
00DCA10C,14.0,18.0,22.00
01FE860A,14.0,19.0,20.78
0336A033,114.0,128.0,135.39
058CD6C0,113.0,124.0,131.00


Computes the final non-parametric safety stock using the formula SS = (Quantile - Average) * sqrt(Lead Time), ensuring the result is non-negative and rounded up.

In [65]:
# Merge the original aggregated stats (df_agg) with the new non-parametric quantiles (df_quantiles_np)
df_ss_np = df_agg.merge(df_quantiles_np, on='sku_number')


df_ss_np['sqrt_lead_time'] = np.sqrt(df_ss_np['avg_lead_time'])

base_ss_75 = (df_ss_np['q_np_75'] - df_ss_np['qty_avg']).clip(lower=0)
base_ss_90 = (df_ss_np['q_np_90'] - df_ss_np['qty_avg']).clip(lower=0)
base_ss_95 = (df_ss_np['q_np_95'] - df_ss_np['qty_avg']).clip(lower=0)

df_ss_np['ss_np_75_raw'] = base_ss_75 * df_ss_np['sqrt_lead_time']
df_ss_np['ss_np_90_raw'] = base_ss_90 * df_ss_np['sqrt_lead_time']
df_ss_np['ss_np_95_raw'] = base_ss_95 * df_ss_np['sqrt_lead_time']

df_ss_np['ss_np_75'] = np.ceil(df_ss_np['ss_np_75_raw']).fillna(0)
df_ss_np['ss_np_90'] = np.ceil(df_ss_np['ss_np_90_raw']).fillna(0)
df_ss_np['ss_np_95'] = np.ceil(df_ss_np['ss_np_95_raw']).fillna(0)
# Display the final non-parametric safety stock values
print("\nNon-Parametric Safety Stock Calculations (Head):")
df_ss_np[['sku_number', 'ss_np_75', 'ss_np_90', 'ss_np_95']].head()


Non-Parametric Safety Stock Calculations (Head):


,sku_number,ss_np_75,ss_np_90,ss_np_95
0,0019425F,71.0,134.0,171.0
1,00DCA10C,16.0,31.0,46.0
2,01FE860A,17.0,36.0,42.0
3,0336A033,73.0,147.0,186.0
4,058CD6C0,76.0,135.0,172.0


No external references were used.